<a href="https://colab.research.google.com/github/Papagoat/algo-trading-strategies/blob/main/Binance_DCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-binance

In [53]:
from binance import AsyncClient, BinanceSocketManager
from binance.client import Client
from datetime import datetime, timedelta
from getpass import getpass
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Binance API Credentials

In [ ]:
api_key = getpass('Enter API KEY here')
api_secret = getpass('Enter API SECRET here')

In [55]:
client = Client(api_key, api_secret)
ticker = 'ETHUSDT'
bm = BinanceSocketManager(client)
ts = bm.trade_socket(ticker)

# Download Binance candlestick charts

In [56]:
klines = client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1DAY, "1 year ago UTC")
df = pd.DataFrame(klines)
df.columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

cols = ['Open time','Close time']
for col in cols:
    df[col] = pd.to_datetime(df[col], unit='ms')

cols_to_convert = ['Open', 'High', 'Low', 'Close']

df[cols_to_convert] = pd.to_numeric(df[cols_to_convert].stack(), errors='coerce').unstack()

cols_to_drop = ['Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
df = df.drop(cols_to_drop, axis = 1)

df['Day'] = df['Open time'].dt.strftime('%d')
df = df[(df['Day'] == '01')]
df = df.reset_index()

# Calculate Profit and Loss

In [57]:
def _sum(arr): 
    sum=0
    for i in arr:
        sum = sum + i
          
    return(sum) 

In [58]:
def buy_sell(df):
  total_amount = []
  total_spent = 0
  for i in range(len(df)):
    amount = 1 / df['Close'][i] * 100
    total_spent += 100
    total_amount.append(amount)
  print(f'Total Spent: {total_spent}')
  print(f'Total {ticker}: {_sum(total_amount)}')

In [59]:
buy_sell(df)

Total Spent: 1200
Total ETHUSDT: 2.004425167450506
